In [ ]:
# Plots for section 5.2 hydrogen technologies

In [ ]:
#TODO: show how it is possibe that less electrolysis capacity produces more hydrogen in EXP (higher utilization / flh)

In [ ]:
# QUESTIONS:
#

### Imports

In [ ]:
import pandas as pd
import matplotlib
import numpy as np
import pypsa
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, LineString
from datetime import datetime
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
import seaborn as sns
from matplotlib.lines import Line2D
import math


# imported own functions
from utils import nodal_balance

# imported own definitions
from utils import carrier_colors, carrier_renaming, resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal
from utils import c_h2_gen, c_h2_con, c_el_gen_s, c_el_con_s

# general variables
font1 = {'fontname':'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/5.2_hydrogen_technologies'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

In [ ]:
# Network imports
stst =pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp =pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

In [ ]:
# stst and exp dataframes
# spatial
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

# temporal
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

In [ ]:
# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger", "solar thermal"]

In [ ]:
# Notebook Functions


In [ ]:
# Regions
onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

## Interaction of technologies

### Hydrogen Balances

**Differences:**
- stst has no 'H2 pipeline', 'H2 pipeline retrofitted'
- exp has no 'H2 Fuel Cell', 'SMR', 'SMR CC'

#### all year

In [ ]:
model = "EXP"
carrier = ["H2"]
loads = ["land transport fuel cell", "H2 for industry", "H2 for shipping"]
loads_not_liquid = ["land transport fuel cell", "H2 for industry"]
period = "2013"
# exclude all technologies that contribute less than th
th = 0.005

# resolution
# "8h", "D", "W", "M"
res = "D"

if model == "STST":
    n = stst
elif model == "EXP":
    n = exp

nb_h2 = nodal_balance(n, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_h2 = nb_h2.unstack(level=[1]) / 1e3
# calc loads
loads_h2 = abs(nb_h2[loads_not_liquid].sum(axis=1))
# drop loads
nb_h2.drop(loads_not_liquid, axis=1, inplace=True)
# rename unhandy column names
nb_h2.rename(columns=carrier_renaming, inplace=True)

####### plot #######

# all year

fig, ax = plt.subplots(figsize=(14, 8))

# resample
df = nb_h2.resample(res).sum()
df_loads = loads_h2.resample(res).sum()

# drop H2
df.drop("H2", axis=1, inplace=True)

# split into df with positive and negative values
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)

df_pos_share = df_pos.sum() / df_pos.sum().sum()
df_pos = df_pos[df_pos_share[df_pos_share > th].sort_values(ascending=False).index]
df_neg_share = df_neg.sum() / df_neg.sum().sum()
df_neg = df_neg[df_neg_share[df_neg_share > th].sort_values(ascending=False).index]
# get colors
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values that are also present on positive side, so that they are not shown and plot negative values
f = lambda c: '_' + c
cols = [f(c) if (c in df_pos.columns) else c for c in  df_neg.columns]
cols_map = dict(zip(df_neg.columns, cols))
ax = df_neg.rename(columns=cols_map).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period]
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses) [$€/MWh_{h2}$]", secondary_y=True)
ax2.grid(False)
# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# plot loads
df_loads.plot(style=":", color="black", label="hydrogen loads\n(without H2 for shipping)")

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.15, 1), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("Total hydrogen balance [$GWh_{h2}$]")
ax.set_xlabel("")
ax.set_title(f"Hydrogen Balance for {period} ({model})", fontsize=16, pad=15,  **font1)
fig.tight_layout()

#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_bal_all_year_{model}.png")

#### January

In [ ]:
model = "STST"
carrier = ["H2"]
loads = ["land transport fuel cell", "H2 for industry", "H2 for shipping"]
loads_not_liquid = ["land transport fuel cell", "H2 for industry"]
period = "2013-01"
# exclude all technologies that contribute less than th
th = 0.005

# resolution
# "8h", "D", "W", "M"
res = "3h"

if model == "STST":
    n = stst
elif model == "EXP":
    n = exp

nb_h2 = nodal_balance(n, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_h2 = nb_h2.unstack(level=[1]) / 1e3
# calc loads
loads_h2 = abs(nb_h2[loads_not_liquid].sum(axis=1))
# drop loads
nb_h2.drop(loads_not_liquid, axis=1, inplace=True)
# rename unhandy column names
nb_h2.rename(columns=carrier_renaming, inplace=True)

####### plot #######

# all year

fig, ax = plt.subplots(figsize=(14, 8))

# resample
df = nb_h2.resample(res).sum()
df_loads = loads_h2.resample(res).sum()

# drop H2
df.drop("H2", axis=1, inplace=True)

# split into df with positive and negative values
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)

df_pos_share = df_pos.sum() / df_pos.sum().sum()
df_pos = df_pos[df_pos_share[df_pos_share > th].sort_values(ascending=False).index]
df_neg_share = df_neg.sum() / df_neg.sum().sum()
df_neg = df_neg[df_neg_share[df_neg_share > th].sort_values(ascending=False).index]
# get colors
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values that are also present on positive side, so that they are not shown and plot negative values
f = lambda c: '_' + c
cols = [f(c) if (c in df_pos.columns) else c for c in  df_neg.columns]
cols_map = dict(zip(df_neg.columns, cols))
ax = df_neg.rename(columns=cols_map).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period]
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses) [$€/MWh_{h2}$]", secondary_y=True)
ax2.grid(False)
# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# plot loads
df_loads.plot(style=":", color="black", label="hydrogen loads\n(without H2 for shipping)")

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.15, 1), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("Total hydrogen balance [$GWh_{h2}$]")
ax.set_xlabel("")
ax.set_title(f"Hydrogen Balance for {period} ({model})", fontsize=16, pad=15,  **font1)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_bal_january_{model}.png")

#### May

In [ ]:
model = "EXP"
carrier = ["H2"]
loads = ["land transport fuel cell", "H2 for industry", "H2 for shipping"]
loads_not_liquid = ["land transport fuel cell", "H2 for industry"]
period = "2013-05"
# exclude all technologies that contribute less than th
th = 0.005

# resolution
# "8h", "D", "W", "M"
res = "3h"

if model == "STST":
    n = stst
elif model == "EXP":
    n = exp

nb_h2 = nodal_balance(n, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_h2 = nb_h2.unstack(level=[1]) / 1e3
# calc loads
loads_h2 = abs(nb_h2[loads_not_liquid].sum(axis=1))
# drop loads
nb_h2.drop(loads_not_liquid, axis=1, inplace=True)
# rename unhandy column names
nb_h2.rename(columns=carrier_renaming, inplace=True)

####### plot #######

# all year

fig, ax = plt.subplots(figsize=(14, 8))

# resample
df = nb_h2.resample(res).sum()
df_loads = loads_h2.resample(res).sum()

# drop H2
df.drop("H2", axis=1, inplace=True)

# split into df with positive and negative values
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)

df_pos_share = df_pos.sum() / df_pos.sum().sum()
df_pos = df_pos[df_pos_share[df_pos_share > th].sort_values(ascending=False).index]
df_neg_share = df_neg.sum() / df_neg.sum().sum()
df_neg = df_neg[df_neg_share[df_neg_share > th].sort_values(ascending=False).index]
# get colors
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values that are also present on positive side, so that they are not shown and plot negative values
f = lambda c: '_' + c
cols = [f(c) if (c in df_pos.columns) else c for c in  df_neg.columns]
cols_map = dict(zip(df_neg.columns, cols))
ax = df_neg.rename(columns=cols_map).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period]
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses) [$€/MWh_{h2}$]", secondary_y=True)
ax2.grid(False)
# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# plot loads
df_loads.plot(style=":", color="black", label="hydrogen loads\n(without H2 for shipping)")

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.15, 1), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("Total hydrogen balance [$GWh_{h2}$]")
ax.set_xlabel("")
ax.set_title(f"Hydrogen Balance for {period} ({model})", fontsize=16, pad=15,  **font1)
fig.tight_layout()

#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_bal_may_{model}.png")

## Focus on technologies

### Load duration curves

In [ ]:
df_stst_ts[[c + "_gen_h2" for c in c_h2_gen]]

In [ ]:
df_exp_ts[[c + "_gen_h2" for c in c_h2_gen]]
df_con = df_stst_ts[[c + "_con_h2" for c in c_h2_con]]

In [ ]:
[c + "_con_h2" for c in c_h2_con]

In [ ]:
# load duration curves of hydrogen generating and consuming technologies

model = "STST"

if model == "STST":
    n = stst
    df = df_stst_ts
    df_gen = df_stst_ts[[c + "_gen_h2" for c in c_h2_gen]]
    df_con = df_stst_ts[[c + "_con_h2" for c in c_h2_con]]
elif model == "EXP":
    n = exp
    df = df_exp_ts
    df_gen = df_exp_ts[['H2 Electrolysis_gen_h2']]
    df_con = df_exp_ts[['H2 liquefaction_con_h2', 'Sabatier_con_h2', 'Fischer-Tropsch_con_h2']]

fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(16, 7))

for i, c in enumerate(df_gen.columns):
    gen = df_gen[c] / 1e3
    gen = pd.DataFrame(gen.sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax1.plot(gen, color = carrier_colors[c.split('_')[0]], label=c.split('_')[0])

# plot load
load = (n.loads_t.p.loc[: , n.loads.carrier.isin(loads_not_liquid)].sum(axis=1) / 1000) * 3
ax1.hlines(y=load.mean(), xmin=0, xmax=1, ls="dashed", linewidth=1, color='saddlebrown', label="hydrogen load (mean)")
ax1.hlines(y=load.max(), xmin=0, xmax=1, ls="dashed", linewidth=1, color='red', label = "hydrogen load (max)")

ax1.set_ylabel("Generation [$GWh_{h2}$]")
ax1.set_xlabel("Fraction of total time")
ax1.set_facecolor("lightgrey")
ax1.legend()
ax1.grid(True)
ax1.set_title(f"Load duration curves of hydrogen generating technologies ({model})", fontsize=16, **font1)

for i, c in enumerate(df_con.columns):
    con = df_con[c] / 1e3
    con = pd.DataFrame(con.sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(con, color = carrier_colors[c.split('_')[0]], label=c.split('_')[0])

ax2.set_ylabel("Generation [$GWh_{h2}$]")
ax2.set_xlabel("Fraction of total time")
ax2.set_facecolor("lightgrey")
ax2.legend(ncol=2)
ax2.grid(True)
ax2.set_title(f"Load duration curves of hydrogen consuming technologies ({model})", fontsize=16, **font1)

fig.tight_layout()
#plt.close()
plt.show()

# fig.savefig(f"{PLOT_DIR}/h2_load_dur_{model}.png")

In [ ]:
# new indices: cut off Fuel cell and SMR completely and SMR CC for exp
c_h2_gen_stst = ['H2 Electrolysis', 'SMR CC']
c_h2_gen_exp = ['H2 Electrolysis']
c_h2_gen = ['H2 Electrolysis', 'SMR CC']
c_h2_con = ['H2 liquefaction', 'Sabatier', 'Fischer-Tropsch']

### Capacity factors

In [ ]:
# delete cf for SMR CC in exp
df_exp_ons["SMR CC_cf_gen_h2"] = pd.DataFrame(np.nan, index=df_exp_ons.index, columns = ["SMR CC_cf_gen_h2"])

##### Boxplot

In [ ]:
# capacity factors across regions as boxplot (STST & EXP)

# boxplot propertes
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_h2_gen = df_stst_ons[[c+ "_cf_gen_h2" for c in c_h2_gen] + [c + "_cf_con_h2" for c in c_h2_con]]
stst_h2_gen = stst_h2_gen.values
exp_h2_gen = df_exp_ons[[c+ "_cf_gen_h2" for c in c_h2_gen]+ [c + "_cf_con_h2" for c in c_h2_con]]
#dh2ete entry for SMR CC
exp_h2_gen = exp_h2_gen.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_h2_gen)
filtered_stst = [d[m] for d, m in zip(stst_h2_gen.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_h2_gen)
filtered_exp = [d[m] for d, m in zip(exp_h2_gen.T, mask_exp.T)]

cf_index = [c+ "_cf_gen_h2" for c in c_h2_gen] + [c + "_cf_con_h2" for c in c_h2_con]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen] + [c + "_con_h2" for c in c_h2_con]
index = c_h2_gen + c_h2_con
ticks = [carrier_renaming.get(n, n) for n in index]
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops,
                        zorder=1
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops,
                       zorder=2
                       )

# generation weighted mean
gwm_cf_stst =np.multiply(df_stst_ons[cf_index], (df_stst_ons[gen_con_index] / df_stst_ons[gen_con_index].sum())).sum()
gwm_cf_exp =np.multiply(df_exp_ons[cf_index], (df_exp_ons[gen_con_index] / df_exp_ons[gen_con_index].sum())).sum()
# set value for SMR CC in exp to nan
gwm_cf_exp["SMR CC_cf_gen_h2"] = np.nan

ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, gwm_cf_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, gwm_cf_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[cf_index].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1.01), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[cf_index].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.xticks(rotation=90)
# plt.title("Market values of electricity generating technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)
ax.grid(True)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
ax.legend([patch1, patch2], ['STST', 'EXP'], loc="lower right")

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_gen_con_cap_fac_STST_EXP.png")

##### Map

In [ ]:
# capacity factors across regions as map

model = "STST"

if model == "STST":
    df = df_stst_ons
    c_h2_gen_index = c_h2_gen_stst

elif model == "EXP":
    df = df_exp_ons
    c_h2_gen_index = c_h2_gen_exp

carriers = c_h2_gen + c_h2_con
col_names = [[c+ "_cf_gen_h2" for c in c_h2_gen_index ] + [c + "_cf_con_h2" for c in c_h2_con]]

cf_index = [c+ "_cf_gen_h2" for c in c_h2_gen_index] + [c + "_cf_con_h2" for c in c_h2_con]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + [c + "_con_h2" for c in c_h2_con]
index = c_h2_gen_index + c_h2_con
ticks = [carrier_renaming.get(n, n) for n in index]

fig, axs = plt.subplots(ncols=2, nrows=math.ceil(len(index)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, math.ceil(len(index)/2) * 6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if i >= len(index):
        ax.axis('off')
        continue

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{cf_index[i]}",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"capacity factors",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[gen_con_index[i]].abs().max()
    # blue if negative and green if positive
    colors = ['darkblue' if (x < 0) else 'darkgreen' for x in df[gen_con_index[i]] ]

    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(df[gen_con_index[i]].abs() / max_size) * 300,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="darkgreen", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="darkblue", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "$TWh_{h2}$" if max_size > 1e3 else "$GWh_{h2}$"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Generation', 'Consumption', f"max circle size:\n{round(max_size)} {unit}"), numpoints=1, loc="upper left", prop={'size': 9})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)


    ax.set_title(f"{ticks[i]}", fontsize=16, **font1)

# fig.suptitle(f"Spatial Differences in the electricity generation of the VRE technologies ({model})", fontsize=16, **font1)
fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}/cap_fac_h2_gen_con_{model}.png")

### Generation and capacity difference

In [ ]:
c_h2_gen_index = c_h2_gen
# remove H2 liquefaction as the capacity and the generation is exactly tehr same in both scenarios
c_h2_con_index = [x for x in c_h2_con if x != 'H2 liquefaction']

cap_index = [c+ "_cap_gen_h2" for c in c_h2_gen_index] + [c + "_cap_con_h2" for c in c_h2_con_index]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + [c + "_con_h2" for c in c_h2_con_index]
index = c_h2_gen_index + c_h2_con_index
ticks = [carrier_renaming.get(n, n) for n in index]

# STST - EXP
for c in cap_index:
    df_stst_ons[f"{c}_STST-EXP"] = df_stst_ons[c] - df_exp_ons[c]

fig, axs = plt.subplots(ncols=2, nrows=math.ceil(len(index)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, math.ceil(len(index)/2) * 6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if i >= len(index):
        ax.axis('off')
        continue

    abs_max = df_stst_ons[f"{cap_index[i]}_STST-EXP"].abs().max()
    #unit_cap = "TW" if abs_max > 1e4 else "GW"
    #df[f"{carriers[i]}_cap_STST-EXP"] = (df[f"{carriers[i]}_cap_STST-EXP"] / 1e3) if abs_max > 1e4 else df[f"{carriers[i]}_cap_STST-EXP"]
    #abs_max = abs_max / 1e3 if abs_max > 1e4 else abs_max

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df_stst_ons.to_crs(crs.proj4_init).plot(column=f"{cap_index[i]}_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"Capacity differences ($GW_{h2}$)",'orientation': "vertical",'shrink' : 0.8}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df_stst_ons[gen_con_index[i]] - df_exp_ons[gen_con_index[i]]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df_stst_ons.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 500,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "$TWh_{h2}$" if max_size > 1e3 else "$GWh_{h2}$"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased generation in STST', 'Increased generation in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{ticks[i]} capacity (STST - EXP)", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the capacity and generation for electricity generating VRE technologies (STST -EXP)", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_cap__STST-EXP_map_all.png")

In [ ]:
df_stst_ts["H2 liquefaction_gen_h2"]

### Generation heatmap

In [ ]:
# Generation heatmap and average

model = "STST"

if model == "STST":
    n = stst
    df_1 = df_stst_ts
    c_h2_gen_index = c_h2_gen_stst
elif model == "EXP":
    n = exp
    df_1 = df_exp_ts
    c_h2_gen_index = c_h2_gen_exp

c_h2_con_index = c_h2_con

cap_index = [c+ "_cap_gen_h2" for c in c_h2_gen_index] + [c + "_cap_con_h2" for c in c_h2_con_index]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + [c + "_con_h2" for c in c_h2_con_index]
index = c_h2_gen_index + c_h2_con_index
ticks = [carrier_renaming.get(n, n) for n in index]


fig, axs = plt.subplots(nrows=math.ceil(len(index)/2), ncols=2, figsize=(18, math.ceil(len(index)/2)*5))

for i, ax in enumerate(axs.reshape(-1)):

    if i >= len(index):
        ax.axis('off')
        continue

    # generation in TWh
    df = df_1[gen_con_index[i]] / 1e3

    hours = df.index.hour.unique()[::-1]
    df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

    for hour in hours:
        df_start[str(hour)] = df[df.index.hour==hour].values

    sns.heatmap(df_start.transpose(),
                ax=ax,
                cmap=plt.get_cmap("magma_r"),
                linewidth=0.001,
                xticklabels=15,
                cbar_kws={'label': 'Generation in GWh', 'pad': 0.1})

    ax.set_title(f"{ticks[i]}", fontsize=16, **font1)
    ax.set_ylabel("hour of the day", fontsize=12, **font1)
    ax.set_xlabel("day of the year", fontsize=12, **font1)

    # Rewrite the y labels
    x_labels = ax.get_xticks()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

    # Plot generation profile on second y-axis
    # resample and transform axis of date to index (heatmap works not with dates)
    df2 = df.resample("5D").sum() / 1e3
    df2 = df2[df2.index.year == 2013]
    df2.index = df2.index.dayofyear - 0.5

    ax2 = ax.twinx()
    ax2.plot(df2, color="white", lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    ax2.legend( loc='lower right', labels=['Generation (right axis)'])
    ax2.set_ylabel("Generation in TWh (5-day sum)", fontsize=12, **font1)
    ax2.grid(False)

fig.tight_layout(pad=3)
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_gen_con_profiles_{model}.png")

## Backup (have look at)

## Generation

In [ ]:
# how long are the electrolyzers running
# load duration curve
# capacity factors
# pie chart of which electricity (from which source?) is used for electrolysers (is that possible)

![](../../../../Pictures/Screenshots/Screenshot_20230215_163455.png)

In [ ]:
# get generation of all carriers
gen_stst = pd.DataFrame(index=stst.generators_t.p.index)
gen_exp = pd.DataFrame(index=exp.generators_t.p.index)

for n, gen in zip([stst, exp], [gen_stst, gen_exp]):

    for carrier in n.generators.carrier.unique():
        gen[carrier]= n.generators_t.p.loc[:, n.generators.carrier == carrier].mean(axis=1)

    for carrier in n.links.carrier.unique():
        gen[carrier]= abs(n.links_t.p1.loc[:, n.links.carrier == carrier]).mean(axis=1)

    for carrier in n.storage_units.carrier.unique():
        gen[carrier]= n.storage_units_t.p.loc[:, n.storage_units.carrier == carrier].mean(axis=1)

gen_stst.head()
gen_exp.head()

In [ ]:
# why is this in a whole other magnitude than capacity?
# does sorting the whole df make sense? shouldtn the be sorted seperately?

df = pd.DataFrame(gen_stst["H2 Electrolysis"].sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
df["H2 Electrolysis no h2"] = pd.DataFrame(gen_exp["H2 Electrolysis"].sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
df.columns = ["no h2 network","with h2 network" ]

plt.figure(figsize=(10, 6))
df.plot()
plt.title("Electrolyzer system load duration curve")
plt.ylabel("Electrolyzer load [MWh]")
plt.xlabel("Fraction of total time")
plt.legend()
plt.show()

## Storage

In [ ]:
stst.links[stst.links.carrier == "H2 Electrolysis"].efficiency.unique()  # Electrolysis: 68 % efficiency
stst.links[stst.links.carrier == "H2 Fuel Cell"].efficiency.unique()  # Fuel Cell: 50 % efficiency
stst.links[stst.links.carrier == "battery charger"].efficiency.unique()  # Fuel Cell: 0.979796 efficiency
stst.links[stst.links.carrier == "battery discharger"].efficiency.unique()  # Fuel Cell: 0.979796 efficiency
# storage losses?

![](../../../../Pictures/Screenshots/Screenshot_20230219_110123.png)
Colbertado

In [ ]:
# H2 storage capcity
index_h2stores_no = stst.stores[stst.stores.carrier == "H2"].index
stst.stores[stst.stores.carrier == "H2"]
# Optimised nominal energy capacity outputed by OPF.
df = stst.stores.e_nom_opt[index_h2stores_no]
df.index = df.index.map(stst.stores.bus).map(stst.buses.location)
onshore_regions["h2_cap_no"] = df
plt.plot(df)
fig = plt.figure(figsize=(12, 8))
crs = ccrs.EqualEarth()

ax = plt.axes(projection=ccrs.EqualEarth())
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')
ax.set_title("h2_cap", fontsize=16, **font1)

onshore_regions.to_crs(crs.proj4_init).plot(column="h2_cap_no",
                                            ax=ax,
                                            cmap=plt.get_cmap("magma_r"),
                                            linewidth=0.05,
                                            edgecolor='grey',
                                            legend=True,
                                            legend_kwds={'label': "MWh",
                                                         'orientation': "vertical"})

plt.show()

In [ ]:
# Why are the hydrogen capacities located there?
# calc Correlation with wind, solar and other power plant capacity (e.g.

#### Feed-in and feed-out pattern

In [ ]:
# Energy as calculated by the OPF.
index_h2stores = stst.stores[stst.stores.carrier == "H2"].index

n.stores_t.p[index_h2stores].sum(axis=1).plot()
plt.show()

#### Storage level over time

In [ ]:
# Energy as calculated by the OPF.
n.stores_t.e[index_h2stores].sum(axis=1).plot()
plt.show()

In [ ]:
# storage level of 10 largest stores
index = n.stores.e_nom_opt[index_h2stores].sort_values(ascending=False).head(10).index

# yearly plot
for store in index:
    fig, ax = plt.subplots()
    n.stores_t.e[store].plot(ax=ax, ylabel=store)

## Coming from Storage and Going to Storage

In [ ]:
#### Where ist the hydrogen that is stored coming from and going to?

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    h2_buses = n.stores[n.stores.carrier == "H2"].bus

    # H2 delivering technologies
    n.links[n.links.bus1.isin(h2_buses)]
    i_h2 = n.links[n.links.bus1.isin(h2_buses)].index
    df = pd.DataFrame(n.links_t.p1[i_h2].sum())
    df["carrier"] = df.index.map(n.links.carrier).values
    result_del = abs(df.groupby("carrier").sum())

    # H2 receiving technologies
    n.links[n.links.bus0.isin(h2_buses)]
    i_h2 = n.links[n.links.bus0.isin(h2_buses)].index
    df = pd.DataFrame(n.links_t.p0[i_h2].sum())
    df["carrier"] = df.index.map(n.links.carrier).values
    result_rec = abs(df.groupby("carrier").sum())

    if n==stst:
        ax_del=axs[0, 0]
        ax_rec=axs[0, 1]
        title_del=f"Delivering H2 to stroage (STST)"
        title_rec=f"Recieving H2 from storage (STST)"

    elif n==exp:
        ax_del=axs[1, 0]
        ax_rec=axs[1, 1]
        title_del=f"Delivering H2 to stroage (EXP)"
        title_rec=f"Delivering H2 to stroage (EXP)"

    # plot delivering
    c = [carrier_colors[col] for col in result_del.index]
    percents = result_del.to_numpy() * 100 / result_del.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result_del.index, percents)]

    patches, texts = ax_del.pie(result_del.values.flatten(), colors=c, startangle=0, labels=labels)
    ax_del.axis('equal')
    ax_del.set_title(title_del, pad=20, fontweight="bold")

    # plot receiving
    c = [carrier_colors[col] for col in result_rec.index]
    percents = result_rec.to_numpy() * 100 / result_rec.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result_rec.index, percents)]

    patches, texts = ax_rec.pie(result_rec.values.flatten(), colors=c, startangle=0, labels=labels)
    ax_rec.axis('equal')
    ax_rec.set_title(title_rec, pad=20, fontweight="bold")

fig.tight_layout(pad=5)
plt.show()

In [ ]:
n = exp
h2_buses = n.stores[n.stores.carrier == "H2"].bus

In [ ]:
# technologies delivering the hydrogen
n.links[n.links.bus1.isin(h2_buses)].carrier.unique()

In [ ]:
# technologies receiving the hydrogen
n.links[n.links.bus0.isin(h2_buses)].carrier.unique()

In [ ]:
# links indices
i_h2 = n.links[n.links.bus0.isin(h2_buses)].index
# generation
df = pd.DataFrame(n.links_t.p0[i_h2].sum())
# carrier
df["carrier"] = df.index.map(n.links.carrier).values

In [ ]:
result = df.groupby("carrier").sum()
result

In [ ]:
# receiving hydrogen from stores
# sign of pipelines are wrong but does not matter as the amounts are the same

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 11))

for n, ax in zip([stst, exp],axs):

    # data
    h2_buses = n.stores[n.stores.carrier == "H2"].bus
    # links of reciever
    n.links[n.links.bus0.isin(h2_buses)]
    # links indices
    i_h2 = n.links[n.links.bus0.isin(h2_buses)].index
    # generation
    df = pd.DataFrame(n.links_t.p0[i_h2].sum())
    # carrier
    df["carrier"] = df.index.map(n.links.carrier).values
    result = abs(df.groupby("carrier").sum())

    c = [carrier_colors[col] for col in result.index]
    percents = result.to_numpy() * 100 / result.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result.index, percents)]

    patches, texts = ax.pie(result.values.flatten(), colors=c, startangle=0, labels=labels)
    ax.axis('equal')
    ax.set_title(f"{n}", pad=20, fontweight="bold")


In [ ]:
carrier = "Li ion"  # "battery"

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    buses = n.stores[n.stores.carrier == carrier].bus

    # H2 delivering technologies
    n.links[n.links.bus1.isin(buses)]
    i_buses = n.links[n.links.bus1.isin(buses)].index
    df = pd.DataFrame(n.links_t.p1[i_buses].sum())
    df["carrier"] = df.index.map(n.links.carrier).values
    result_del = abs(df.groupby("carrier").sum())

    # H2 receiving technologies
    n.links[n.links.bus0.isin(buses)]
    i_buses = n.links[n.links.bus0.isin(buses)].index
    df = pd.DataFrame(n.links_t.p0[i_buses].sum())
    df["carrier"] = df.index.map(n.links.carrier).values
    result_rec = abs(df.groupby("carrier").sum())

    if n == stst:
        ax_del = axs[0, 0]
        ax_rec = axs[0, 1]
        title_del = f"Delivering {carrier} to stroage (STST)"
        title_rec = f"Recieving {carrier} from storage (STST)"

    elif n == exp:
        ax_del = axs[1, 0]
        ax_rec = axs[1, 1]
        title_del = f"Delivering {carrier} to stroage (EXP)"
        title_rec = f"Delivering {carrier} to stroage (EXP)"

    # plot delivering
    c = [carrier_colors[col] for col in result_del.index]
    percents = result_del.to_numpy() * 100 / result_del.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result_del.index, percents)]

    patches, texts = ax_del.pie(result_del.values.flatten(), colors=c, startangle=0, labels=labels)
    ax_del.axis('equal')
    ax_del.set_title(title_del, pad=20, fontweight="bold")

    # plot receiving
    c = [carrier_colors[col] for col in result_rec.index]
    percents = result_rec.to_numpy() * 100 / result_rec.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result_rec.index, percents)]

    patches, texts = ax_rec.pie(result_rec.values.flatten(), colors=c, startangle=0, labels=labels)
    ax_rec.axis('equal')
    ax_rec.set_title(title_rec, pad=20, fontweight="bold")

fig.tight_layout(pad=5)
plt.show()

In [ ]:
#### How much of the produced hydrogen is being stored and how much is directly used by consuming technologies?

## Ideas:
- Calc capacity factors / system load duration curves for electrolysis in different regions and try to investigate in differences (amount of wind generation, solar) Is only excess electrolysis consumed by electrolysis?
- Try to determine connection between electrolysis and excess solar / wind production
- Try to determine percent / amount of excess electricity that is used to produce hydrogen. (Basically all energy from renewables that went into electrolysis?)
- pie chart of percentage of electricity that is used directly, used for Electrolysis, battery, ... (startup script)
- Reproduce graph from Victoria, Zhu et al. 2019 – The role of storage technologies (in hydrogen word document); try to find patterns within the charging of battery and the charging with hydrogen (electrolysis -> fuel cell)
-